**NOTE THAT THIS NOTEBOOK WAS RUN IN GOOGLE COLAB AS SUCH CERTAIN IMPORT STATEMENTS MAY DIFFER**

In [0]:
import tensorflow as tf
from tensorflow.keras import Model
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical 

from keras import backend as K
from sklearn.metrics import f1_score

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import pandas as pd
train_path=os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '/content/drive/My Drive/datasets/train.csv'))
training_df = pd.read_csv(train_path, index_col="tripid")

In [0]:
training_df = training_df.fillna(0)

In [0]:
training_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,0.0,0.0,0.0,0.0000,0.0,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,correct
213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,correct
213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,correct


In [0]:
training_df = training_df.replace({'label': {'incorrect': 0, 'correct' : 1}})

In [0]:
training_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,1
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,1
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,1
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,1
189128020,0.0,0.0,0.0,0.0000,0.0,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,1
213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,1
213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1


In [0]:
training_columns = ['duration','meter_waiting','meter_waiting_fare','fare','additional_fare']

In [0]:
target_column = ['label']

In [0]:
training_df[training_columns] = training_df[training_columns]/training_df[training_columns].max()

In [0]:
X = training_df[training_columns].values
y = training_df[target_column].values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [0]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [0]:
count_classes = y_test.shape[1]
print(count_classes)

2


In [0]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [0]:
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=5))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['acc',f1_m,precision_m,recall_m, f1])

In [0]:
model.fit(X_train, y_train, epochs=100, class_weight={0: 5.,1: 1.})

Epoch 1/100
12023/12023 [==============================] - 1s 70us/step - loss: 0.8844 - acc: 0.8979 - f1_m: 0.8978 - precision_m: 0.8978 - recall_m: 0.8978 - f1: 0.4976
Epoch 2/100
12023/12023 [==============================] - 1s 58us/step - loss: 0.8600 - acc: 0.8809 - f1_m: 0.8809 - precision_m: 0.8809 - recall_m: 0.8809 - f1: 0.5551
Epoch 3/100
12023/12023 [==============================] - 1s 56us/step - loss: 0.8517 - acc: 0.8770 - f1_m: 0.8770 - precision_m: 0.8770 - recall_m: 0.8770 - f1: 0.5641
Epoch 4/100
12023/12023 [==============================] - 1s 58us/step - loss: 0.8388 - acc: 0.8636 - f1_m: 0.8636 - precision_m: 0.8636 - recall_m: 0.8636 - f1: 0.5801
Epoch 5/100
12023/12023 [==============================] - 1s 57us/step - loss: 0.8335 - acc: 0.8650 - f1_m: 0.8650 - precision_m: 0.8650 - recall_m: 0.8650 - f1: 0.5844
Epoch 6/100
12023/12023 [==============================] - 1s 57us/step - loss: 0.8267 - acc: 0.8733 - f1_m: 0.8733 - precision_m: 0.8733 - recall_m: 

In [0]:
y_train

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [0]:
pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))

Accuracy on training data: 0.9494302868843079% 
 Error on training data: 0.05056971311569214
Accuracy on test data: 0.9452745914459229% 
 Error on test data: 0.05472540855407715


In [0]:
pred_test

array([[0.038802  , 0.96119803],
       [0.01078611, 0.9892139 ],
       [0.76824564, 0.23175438],
       ...,
       [0.02719227, 0.97280777],
       [0.01313569, 0.98686427],
       [0.00976335, 0.9902367 ]], dtype=float32)